# 1. Instalção e importe de bibliotecas úteis

In [ ]:
%pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [2]:
import os
import cv2
import time
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt

# 2. Obtenção dos Pontos de Articulação por método Holístico pelo Mediapipe

In [3]:
mp_holistic = mp.solutions.holistic # Modelo holístico
mp_drawing = mp.solutions.drawing_utils # Desenhando o modelo

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Conversão das cores de Azul, Verde, Vermelho para Vermelho, Verde, Azul
    image.flags.writeable = False                  # Imagem não é mais gravável
    results = model.process(image)                 # Faz a predição da imagem
    image.flags.writeable = True                   # Imagem torna-se gravável
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Conversão das cores de Vermelho, Verde, Azul para Azul, Verde, Vermelho
    return image, results

In [5]:
def draw_landmarks(image, results):
    # Desenhando as conexões da rosto
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                            )
    # Desenhando as conexões da postura
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=3),
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                            )
    # Desenhando as conexões da mão esquerda
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=3),
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                            )  
    # Desenhando as conexões da mão direita
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,22,10), thickness=1, circle_radius=3),
                            mp_drawing.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=1)
                            ) 

In [9]:
cap = cv2.VideoCapture(0)
# Definindo o modelo do Mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Lendo o retorno da chamada
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)

        # Desenhando landmarks
        draw_landmarks(image, results)

        # Mostrando na tela
        cv2.imshow('OpenCV feed', image)

        # Desligando a tela sem quebrar o código (aperte 'q para fechar')
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [7]:
cap.release()
cv2.destroyAllWindows()

# 3. Obtendo valor dos Pontos de Articulação

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, right_hand, left_hand])

# 4. Definindo pastas de coleta de dados

In [39]:
# Caminho para exportar os dados dos arrays do numpy
DATA_PATH = os.path.join('MP_Data')

# Sinais a serem detectados
actions = np.array(['OBRIGADO', 'OI', 'DESCULPAR', 'POR-FAVOR', 'ENTENDER'])

# Trinta videos que servem para serem dados gravados
no_sequences = 11

# Os videos terão 30 quadros de duração
sequence_length = 10

In [40]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Coletando Pontos de Articulação para teste e treinamento

In [32]:
cap = cv2.VideoCapture(0)
# Definindo o modelo do Mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:

    # Repetição dos sinais
    for action in actions:
        # Repetição da quantidade de videos
        for sequence in range(no_sequences):
            # Repetição dos quadros a serem capturados
            for frame_num in range(sequence_length + 1):

                # Lendo o retorno da chamada
                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)

                # Desenhando landmarks
                draw_landmarks(image, results)

                # Lógica da sequência de abertura da câmera
                if frame_num == 0:
                    cv2.putText(image, 'INICIANDO COLETA DO SINAL {}'.format(action), (10, 100),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
                    
                    # Abrindo a tela
                    cv2.imshow('Tela OpenCV', image)
                    cv2.waitKey(3000)
                else:
                    cv2.putText(image, 'Coletando quadros para {} Video numero {}'.format(action, sequence + 1), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    # Abrindo a tela
                    cv2.imshow('Tela OpenCV', image)
                
                    # Exportando Pontos de Articulação
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num - 1))
                    np.save(npy_path, keypoints)

                # Desligando a tela sem quebrar o código (aperte 'q para fechar')
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

# 6. Preprocessamento dos dados e criação das ferramentas e rótulos

In [23]:
#array de objetos classificadores
label_map = {label: num for num, label in enumerate(actions)}

In [24]:
#sequenciamento dos pontos de articulação e suas respectivas classificações
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), '{}.npy'.format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [25]:
label_map

{'OBRIGADO': 0}

In [26]:
labels

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [30]:
for item in sequences[0][0]:
    print(item)

0.5413862466812134
0.4916648864746094
-0.5498536825180054
0.5582786202430725
0.4471205770969391
-0.5075615048408508
0.569869875907898
0.45112839341163635
-0.5074800252914429
0.581598699092865
0.4550154507160187
-0.5074415802955627
0.5209013223648071
0.43673816323280334
-0.5078941583633423
0.507408857345581
0.43590742349624634
-0.5079450011253357
0.4941732883453369
0.43735212087631226
-0.5085605382919312
0.6004582047462463
0.4854258596897125
-0.2513140141963959
0.47272032499313354
0.4669741094112396
-0.23997862637043
0.5662972331047058
0.555870771408081
-0.45429331064224243
0.5194064378738403
0.5448045134544373
-0.4579019546508789
0.7054553627967834
0.7472794055938721
-0.09153889864683151
0.34501272439956665
0.734795868396759
-0.17427334189414978
0.8210455775260925
1.1673625707626343
-0.49251899123191833
0.1865829974412918
0.9806407690048218
-0.5974395275115967
0.7419016361236572
0.8232638835906982
-0.9546142816543579
0.33422422409057617
0.5570451617240906
-1.0537720918655396
0.69474685

# Em Progresso

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
X = np.array(sequences)

In [ ]:
Y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.05)

## 7. Construindo e treinando uma Rede Neural LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=2000, callbacks=[tb_callback])

## 8. Salvando o modelo

In [ ]:
model.save('action-A-E-I-O-U.h5')

In [ ]:
model.load_weights('action.h5')

## 9. Validando o modelo usando Matriz de Confusão e Acuracidade

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_train)

In [ ]:
ytrue = np.argmax(Y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

## 10. Testando em tempo real

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0,85+num*40),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame

In [ ]:
# 1. Novas variáveis de detecção
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Definindo o modelo do Mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Lendo o retorno da chamada
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)

        # Desenhando landmarks
        draw_landmarks(image, results)
        
        # 2. lógica de previsão
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))

        # 3. lógica de visualização
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Mostrando na tela
        cv2.imshow('OpenCV feed', image)

        # Desligando a tela sem quebrar o código (aperte 'q para fechar')
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()